In [13]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [14]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [15]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [16]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['BCI', 'COUP', 'JJC', 'LHCG', 'SBNY', 'SIVB', 'STOR', 'UMPQ'], dtype='object')


In [17]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ANSS', 'AVAV', 'BANR', 'BCI', 'BGX', 'BPT', 'BWA', 'CEM', 'COO',
       'COUP', 'CW', 'FDP', 'FRA', 'GPRK', 'HEI', 'JJC', 'JLL', 'LHCG', 'ODP',
       'SBNY', 'SIVB', 'WERN'],
      dtype='object')


In [18]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['CAJ', 'CHK', 'COUP', 'JJC', 'LHCG', 'RCII', 'RFP', 'SBNY', 'SIVB',
       'SJI', 'STOR', 'UMPQ'],
      dtype='object')


In [19]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

combined symbols with no volume, same volume and no close, inculdes duplicate symbols: 42
unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 29


In [20]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:       8
symbols with same volume:    22
symbols with no close:       12

symbols total before drop:                                        1,319
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):    29

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,290    1,500
df Close (df_c) after dropped symbols:      1,290    1,500


In [21]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [22]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [23]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-03-10  139.190002  139.440002  134.710007  135.720001  1896000.0   
2023-03-13  135.550003  138.279999  134.759995  136.690002  1771400.0   
2023-03-14  138.440002  139.600006  136.429993  138.399994  1255400.0   
2023-03-15  135.929993  135.929993  132.199997  134.029999  2016700.0   
2023-03-16  133.970001  137.050003  132.369995  136.820007  1745154.0   

                   AA                                            ...  \
                 Open       High        Low      Close   Volume  ...   
Date                                                             ...   
2023-03-10  48.119999  48.119999  44.459999  44.910000  6190400  ...   
2023-03-13  43.509998  46.049999  42.244999  44.709999  5585500  ...   
2023-03-14  45.830002  46.230000  43.919998  44.590000  4197000  ...   
2023-03-15  42.330002  42.470001  38.849998  39.770000  9434500  ...   
2023-03-16  39.169998  41.098301  38.509998  40.750000  6457907  ...   

                 ZUMZ                                                ZWS  \
                 Open       High        Low  Close     Volume       Open   
Date                                                                       
2023-03-10  20.500000  22.360001  20.190001  21.65  1466400.0  22.180000   
2023-03-13  21.250000  21.250000  20.350000  20.66   764900.0  21.000000   
2023-03-14  21.120001  21.320000  20.030001  20.43   626600.0  21.559999   
2023-03-15  20.059999  20.270000  19.280001  19.50   654000.0  20.910000   
2023-03-16  19.160000  19.780001  18.760000  19.25   528822.0  20.690001   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-03-10  22.250000  21.230000  21.309999  1717200.0  
2023-03-13  21.590000  20.910000  21.070000  1581400.0  
2023-03-14  21.730000  20.900000  21.379999  1440900.0  
2023-03-15  21.110001  20.540001  20.980000  1605400.0  
2023-03-16  21.139999  20.584999  20.969999  1474346.0  

[5 rows x 6450 columns]

In [24]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AAWW,AB,ABB,ABBV,ABC,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-10,135.720001,44.910000,15.46,128.039993,148.500000,99.930000,36.869999,33.139999,149.710007,149.600006,...,28.440001,123.529999,288.190002,73.809998,38.650002,40.349998,25.740000,161.529999,21.65,21.309999
2023-03-13,136.690002,44.709999,14.85,124.360001,150.470001,99.519997,36.009998,32.919998,151.949997,148.369995,...,27.620001,124.250000,284.920013,73.239998,39.459999,29.969999,25.709999,163.410004,20.66,21.070000
2023-03-14,138.399994,44.590000,14.66,121.699997,152.589996,99.300003,35.709999,33.759998,153.850006,149.289993,...,27.469999,126.629997,292.459991,74.370003,39.610001,31.309999,26.270000,164.559998,20.43,21.379999
2023-03-15,134.029999,39.770000,13.86,121.769997,152.990005,102.449997,35.080002,31.990000,154.059998,149.610001,...,27.070000,125.260002,287.739990,73.779999,39.919998,30.709999,25.940001,163.570007,19.50,20.980000
2023-03-16,136.820007,40.750000,14.12,122.029999,155.850006,102.459999,35.310001,32.549999,155.300003,152.050003,...,28.000000,125.430000,294.929993,74.510002,40.849998,32.110001,28.000000,166.309998,19.25,20.969999
